In [9]:
import os

In [10]:
os.chdir("../.")

In [11]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class PrepareCallbacksConfig:
    root_dir: Path
    tensorboard_root_log_dir: Path
    checkpoint_model_filepath: Path




In [12]:
# create the configuration manager
# now i will write my configuration
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories

class ConfigurationManager:
    def __init__(
            self,
            config_filepath = CONFIG_FILE_PATH,
            params_filepath = PARAMS_FILE_PATH):
            self.config = read_yaml(config_filepath)
            self.params = read_yaml(params_filepath)

            create_directories([self.config.artifacts_root])

    def get_prepare_callbacks_config(self) -> PrepareCallbacksConfig:
        config = self.config.prepare_callbacks
        model_ckpt_dir = os.path.dirname(config.checkpoint_model_filepath)
        create_directories([
             Path(model_ckpt_dir),
             Path(config.tensorboard_root_log_dir)
        ])

        prepare_callback_config = PrepareCallbacksConfig(
            root_dir = Path(config.root_dir),
            tensorboard_root_log_dir = Path(config.tensorboard_root_log_dir),
            checkpoint_model_filepath = Path(config.checkpoint_model_filepath)
             
        )
        return prepare_callback_config

In [13]:
# components


import os
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf
import time

class PrepareCallback:
    def __init__(self,config:PrepareCallbacksConfig):
        self.config = config

    @property
    def _create_tb_callbacks(self):
        timestamp = time.strftime("%Y-%m-%d-%H-%M-%S")
        tb_running_log_dir = os.path.join(
            self.config.tensorboard_root_log_dir,
            f"tb_logs_at_{timestamp}"
        )  
        return tf.keras.callbacks.TensorBoard(log_dir=tb_running_log_dir)

    @property
    def _create_ckpt_callbacks(self):
        return tf.keras.callbacks.ModelCheckpoint(
            filepath=self.config.checkpoint_model_filepath,
            save_best_only=True
        )

    def get_tb_ckpt_callbacks(self):
        return[
            self._create_tb_callbacks,
            self._create_ckpt_callbacks
        ]

In [16]:
# prepare the pipeline
try:
    config = ConfigurationManager()
    prepare_callbacks_config= config.get_prepare_callbacks_config()
    prepare_callbacks = PrepareCallback(config=prepare_callbacks_config)
    callback_list = prepare_callbacks.get_tb_ckpt_callbacks()

except Exception as e:
    raise e

--- Logging error ---
Traceback (most recent call last):
  File "c:\Program Files\Python312\Lib\logging\__init__.py", line 464, in format
    return self._format(record)
           ^^^^^^^^^^^^^^^^^^^^
  File "c:\Program Files\Python312\Lib\logging\__init__.py", line 460, in _format
    return self._fmt % values
           ~~~~~~~~~~^~~~~~~~
KeyError: 'messages'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "c:\Program Files\Python312\Lib\logging\__init__.py", line 1160, in emit
    msg = self.format(record)
          ^^^^^^^^^^^^^^^^^^^
  File "c:\Program Files\Python312\Lib\logging\__init__.py", line 999, in format
    return fmt.format(record)
           ^^^^^^^^^^^^^^^^^^
  File "c:\Program Files\Python312\Lib\logging\__init__.py", line 706, in format
    s = self.formatMessage(record)
        ^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Program Files\Python312\Lib\logging\__init__.py", line 675, in formatMessage
    return

AttributeError: 'PrepareCallback' object has no attribute 'get_tb_ckpt_callbacks'